<a href="https://colab.research.google.com/github/raheelam98/LangGraph/blob/main/langgraph_code_documents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**rm note**

state:- structure of work flow

memory :- presisten memory (store things)

API :- talk one meachine to another meachine

In [ ]:
# check it later
# module 1 : 2.2_chains_reducers.ipynb

llm_with_tools: ChatGoogleGenerativeAI = llm.bind_tools([multiply])

  ## langchain-academy

[LangChain Academy](https://academy.langchain.com/courses/intro-to-langgraph)
  
[langchain-academy - Github](https://github.com/langchain-ai/langchain-academy/tree/main)

In [ ]:
# Install the required packages:
%%capture --no-stderr
%pip install -U langgraph langsmith # check
%pip install --quiet -U langchain_google_genai langchain_core langgraph

**Description of Libraries**
* `langchain-google-genai:` LangChain doesn't have its own chat model, so we use Google's Gemini.
* `langchain-core`: Utilizes LangChain core libraries, e.g., human-message, ai-message.
* `langchain_community:` Tavily-python is part of the LangChain community package.
* `tavily-python: `Used for web search.
Note: Use both langchain_community and tavily-python together.


In [ ]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "project_name"

In [ ]:
# API Keys
# Get the GEMINI API key from user data
from google.colab import userdata
gemini_api_key = userdata.get('GEMINI_API_KEY')

We'll use [LangSmith](https://docs.smith.langchain.com/) for [tracing](https://docs.smith.langchain.com/concepts/tracing).

In [ ]:
from google.colab import userdata
LANGCHAIN_API_KEY= userdata.get('LANGCHAIN_API_KEY')

In [ ]:
# Initialize the ChatGoogleGenerativeAI with the Gemini model
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    max_retries=2,
    api_key=gemini_api_key
)

In [ ]:
# Invoke the LLM with a query
result = llm.invoke("Rain is expected in Karachi starting today?")
result

# Detail of each module

### **Moduel 0**

[Class-01: Mastering LangGraph In a New Way: Core Concepts & Implementation Node, Edges, States - Nov 8, 2024](https://www.youtube.com/watch?v=jIX9P12IkQM)

#### **Description of LLM methods**
* `stream`: Stream back chunks of the response (useful for streaming results).
* `invoke`: Call the chain on an input (provides a complete result from start to end).

#### **Search Tools - TAVILY**

* TAVILY is a search engine created by the LangChain open community. It provides the latest accurate data from the web.
* LLMs do not have access to real-time information, e.g., weather forecasts.
* LLMs do not have the latest information. That's why we use TAVILY to access real-time information, e.g., weather forecasts.


In [ ]:
import os
from google.colab import userdata

# Get the TAVILY API key from user data and set the environment variable
TAVILY_API_KEY = userdata.get('TAVILY_API_KEY')
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

### **Module 1 :** 2.2_chains_reducers.ipynb

[Class-02: Mastering LangGraph In a New Way: LLM with Tool Calling, Chains & Reducers - Nov 9, 2024 ](https://www.youtube.com/watch?v=g1GqJx2k1Hk)


### Using messages as state

With these foundations in place, we can now use [`messages`](https://python.langchain.com/v0.2/docs/concepts/#messages) in our graph state.

Let's define our state, `MessagesState`, as a `TypedDict` with a single key: `messages`.

`messages` is simply a list of messages, as we defined above (e.g., `HumanMessage`, etc).

In [ ]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage

class MessagesState(TypedDict):
    messages: list[AnyMessage]

Reducers address the issue of **new state values overriding prior ones**. By using the pre-built **`add_messages` reducer**, we ensure that **messages are appended to the existing list** rather than replaced. We just need to **annotate the messages key with this reducer function as metadata**.

```bash
 messages: Annotated[list[AnyMessage], add_messages]
 ```

In [ ]:
# Import necessary modules for typing and LangGraph
from typing import Annotated
from langgraph.graph.message import add_messages

# Define a TypedDict class 'MessagesState' to store messages
class MessagesState(TypedDict):
    # Annotate the 'messages' key to use the 'add_messages' reducer function for appending messages
    messages: Annotated[list[AnyMessage], add_messages]


### **Module 1 :**  5_agent_memory.ipynb

[Class-04: Mastering LangGraph In a New Way: Memory, RAG, Fine Tuning, GraphRAG, AI Agents - Nov 15, 2024  ](https://www.youtube.com/watch?v=Zf2C8A6RmJE)

In [ ]:
# Import necessary modules from langgraph and langchain_core
from langgraph.graph import MessagesState
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# # System message (use to increase performance)
# Define a system message to establish the assistant's role
# This message sets the context for the assistant's behavior
sys_msg = SystemMessage(content="You are a helpful assistant tasked with performing arithmetic on a set of inputs.")

# # Node (create assistant node)
# Define a function named 'assistant' that takes a MessagesState as input
def assistant(state: MessagesState) -> MessagesState:
    # Invoke the LLM with tools using the system message and the current state of messages
    # The response is returned in a dictionary format, appended to the 'messages' key
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"]) ] }

    # Main :- llm_with_tools.invoke([sys_msg] + state["messages"])
    # ensures the assistant uses both the predefined context and the conversation history to respond appropriately

    # step 1 : System Message Inclusion: :- llm_with_tools.invoke([sys_msg]
    # calls the language model with just the system message, which establishes the context for the assistant.

    # step 2 : Combined Messages Processing :-  llm_with_tools.invoke(state["messages"])
    # processes the current conversation history (state messages)
    # by including them along with the system message to generate a coherent
    # response considering both the context and the history.


**rm notes**

```bash
from langgraph.graph import MessagesState
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# # System message (create system message for better performance)
sys_msg = SystemMessage(content="You message.")
```

Node (create assistant node)

**Functionality :** Takes input, combines it with the system message, sends them to the LLM, and stores the LLM's response.

When the function gets the prompt, it sends it to the LLM along with both the system message and the prompt


```bash
def assistant(state: MessagesState) -> MessagesState:
    return {"messages": [llm_with_tools.invoke([sys_msg] + state["messages"])]}
```

**Detail**

Define a function named 'assistant' that takes a MessagesState as input

**`def assistant(state: MessagesState) -> MessagesState:`**

This code calls the language model with both the system message and the current conversation history to generate a relevant response.

**`[llm_with_tools.invoke([sys_msg] + state["messages"]) ]`**


**Combined Messages Processing :**
**`[llm_with_tools.invoke([sys_msg] + state["messages"])]`**

This step makes sure the assistant takes into account both the initial instructions and the entire conversation so far to respond appropriately.

* **Including the System Message : `(sys_msg)`** Adding the initial instruction **`(sys_msg)`** to the list of messages. This sets the context for the assistant.

* **Processing Conversation History : `state["messages"]`** Combining the system message with the existing conversation history (all previous messages in **`state["messages"]`**). This ensures the assistant understands the context and history to generate a relevant and coherent response.

  

**rm note**

**Don't retain memory from the initial chat!** because it don't has memory only hase state

When the graph starts and runs, after finishing, the graph state is lost because it gets overwritten.

**Transient State :** Starts a new chat session.

**Steady State :** Remembers details from earlier conversations


**rm notes**


To retain conversations, LangGraph uses **`persistent checkpointing`**.

```bash
react_graph_memory: CompiledStateGraph = builder.compile(checkpointer=memory)
```

LangGraph automatically saves the state after each step. When the graph is invoked again using the same **`thread_id`**, it loads its saved state, allowing the chatbot to continue from where it left off

**Note : Sequential tasks in LangGraph :**

**LangGraph only performs parallel tasks**. To make sequential tasks, we have to design the architecture to make it sequential

**State**

**Note : Checkpoints are Saved in a thread:**

Threads act as memory locations where the state is saved.

Through the **`thread_id`**, the state is saved into the memory location.

**Note : MemorySaver :**

MemorySaver retains the state while connected. Once disconnected, the data is lost.


This means the memory is not persistent and does not save the data when the connection is terminated. For persistent memory, you would need to implement a different solution. (module 2 )

In [ ]:
# Steady State : Remembers details from earlier conversations
from langgraph.checkpoint.memory import MemorySaver
memory: MemorySaver = MemorySaver()
react_graph_memory: CompiledStateGraph = builder.compile(checkpointer=memory)

**Module 1 :**  5_agent_memory.ipynb

**Tutorial**

When we use memory, we need to specify a `thread_id`.

This **`thread_id`** will store our memory location of graph states.

Here is a cartoon:

* The checkpointer write the state at every step of the graph
* These checkpoints are saved in a thread
* We can access that thread in the future using the `thread_id`

![state.jpg](https://cdn.prod.website-files.com/65b8cd72835ceeacd4449a53/66e0e9f526b41a4ed9e2d28b_agent-memory2.png)


In [ ]:
# Specify a thread
config = {"configurable": {"thread_id": "1"}}

In [ ]:
# Specify an input
# Create a list of messages with a HumanMessage containing the content "Add 3 and 4."
messages = [HumanMessage(content="Add 3 and 4.")]

# Run
# Invoke the graph with the specified messages and configuration
messages = react_graph_memory.invoke({"messages": messages}, config)

# Iterate through the resulting messages
for m in messages['messages']:
    # Print each message in a formatted ma
    m.pretty_print()